In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [189]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [190]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
sales_train = sales_train[sales_train['year'].isin([2013,2014]) == False]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [191]:
# Якутск Орджоникидзе, 56
sales_train.loc[sales_train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales_train.loc[sales_train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales_train.loc[sales_train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [192]:
max_sales=1000
sums = sales_train.groupby('item_id')['item_cnt_day'].sum().reset_index().rename(columns={"item_cnt_day":"item_total_sales"}).sort_values(by='item_total_sales')

ids_reject = sums[(sums['item_total_sales'] > 0) & (sums['item_total_sales'] < max_sales)]['item_id'].unique()

In [196]:
train_item_ids = sales_train['item_id'].unique()
train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))
all_shop_ids = np.setdiff1d(all_shop_ids, closed_shop_ids)

In [197]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    #item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    item_ids = all_item_ids
    #dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    dbn_combos = list(product(all_shop_ids, item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [256]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['shop_item_cnt_block'] = training['shop_item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [257]:
ys = sales_train.groupby(['item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = training.merge(ys, on=['item_id', 'date_block_num'], how='left').fillna(0)

ys = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_cnt_block"})

training = training.merge(ys, on=['shop_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"category_cnt_block"})


training = training.merge(ys, on=['item_category_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['shop_id', 'item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_category_cnt_block"})

training = training.merge(ys, on=['shop_id', 'item_category_id', 'date_block_num'], how='left').fillna(0)


In [258]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')


In [207]:
training["shop_cat"] = training["shop_id"].astype(str) + "_" + training["item_category_id"].astype(str)
training["shop_item"] = training["shop_id"].astype(str) + "_" + training["item_id"].astype(str)

In [259]:
a = training.groupby(['shop_id','date_block_num'])['shop_cnt_block'].sum()\
                        .reset_index()
a = a[a['shop_cnt_block'] == 0]
closed_shop_ids = a["shop_id"].unique()
print(closed_shop_ids)


#with pd.option_context('display.max_rows', 999):
    #print(a[a['item_cnt_block'] == 0])

training = training[~training['shop_id'].isin(closed_shop_ids)]

[ 9 17 20 27 29 30 33 36 40 43 51 54]


In [255]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
#columns = ["item_id", "shop_id", "item_category_id", "month", "shop_cat", "shop_item", "date_block_num"]
columns = ["item_id", "shop_id", "item_category_id"]




y_train = training["shop_item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    for column in columns:
        means = training.iloc[in_fold_index].groupby(column)['shop_item_cnt_block'].mean()
            #x_validation[column + "_mean_target"] = means\
        name = column + '_mean_encoding'
        training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index][column].map(means)
    i+=1

fold 1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


KeyError: '[786433, 1048578, 1048579, 786435, 1835012, 1048582, 524294, 1572872, 786437, 262154, 1572874, 524299, 1572875, 1572876, 1048601, 786462, 786463, 786465, 1310756, 1310758, 1310759, 524326, 524328, 262187, 262189, 1048631, 1048632, 1048634, 1048635, 60, 61, 62, 1048639, 66, 1835074, 68, 1835079, 1835093, 1310812, 1835103, 524385, 1048674, 1048675, 1048676, 524389, 1048678, 1835109, 1835106, 1835121, 524403, 1835124, 1835126, 2097271, 1572984, 2097274, 123, 262266, 1048701, 1048702, 127, 262268, 129, 262269, 1310846, 1310847, 1048709, 1310848, 524417, 524418, 524430, 524431, 1835150, 1835152, 1835154, 1835156, 524438, 262295, 524439, 1835159, 154, 262298, 786586, 524462, 2097330, 524468, 524469, 2097334, 1310903, 1835190, 1310906, 2097339, 1835194, 1310909, 1835196, 1835199, 1310922, 1310923, 1048783, 1310928, 1048788, 1048789, 1310934, 215, 524500, 1835225, 1310952, 1048810, 1048811, 1048813, 238, 1048814, 1310957, 1310958, 1573106, 524531, 786676, 524536, 524537, 261, 131098

In [210]:
training.head()

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,shop_cat,shop_item,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding
0,30,2,24,0,40,14.0,987.0,15109.0,57.0,1,2_40,2_30,5.920290,12.705839,8.740981
1,30,2,25,0,40,14.0,907.0,12100.0,68.0,2,2_40,2_30,5.920290,12.705839,8.740981
2,30,2,26,0,40,5.0,762.0,13084.0,68.0,3,2_40,2_30,6.055427,12.517935,8.702927
3,30,2,27,0,40,4.0,859.0,10683.0,83.0,4,2_40,2_30,5.920290,12.705839,8.740981
4,30,2,28,0,40,5.0,843.0,9208.0,58.0,5,2_40,2_30,5.921986,12.423669,8.716483


In [260]:
def add_block_units_stats(df, cols, name):
    print(name)
    name_units = name + '_units'
    name_mean = name + '_mean'
    name_median = name + '_median'
    name_max = name + '_max'
    name_min = name + '_min'
    name_std = name + '_std'
    
    
    try:
        df.drop(columns=[name_units, name_mean, name_median],inplace=True)
    except:
        pass

    
    block_units = df.groupby(cols)[name].sum()\
                        .reset_index().rename(columns={name:name_units})
    df = df.merge(block_units, on=cols, how='left')
    df[name_units].fillna(0,inplace=True)
    df[name_units] = pd.to_numeric(df[name_units].astype(int),downcast='unsigned')
    del block_units
    
    block_units_med = df.groupby(cols)[name].median()\
                        .reset_index().rename(columns={name:name_median})
    print(block_units_med.head())
    df = df.merge(block_units_med, on=cols, how='left')
    df[name_median].fillna(0,inplace=True)
    #df[name_median] = pd.to_numeric(df[name_median].astype(int),downcast='unsigned')
    del block_units_med
    
    block_means = df.groupby(cols)[name].mean()\
                        .reset_index().rename(columns={name:name_mean})
    df = df.merge(block_means, on=cols, how='left')
    df[name_mean].fillna(0,inplace=True)
    #df[name_mean] = pd.to_numeric(df[name_mean],downcast='float')
    del block_means
    
    block_max = df.groupby(cols)[name].max()\
                        .reset_index().rename(columns={name:name_max})
    df = df.merge(block_max, on=cols, how='left')
    df[name_max].fillna(0,inplace=True)
    #df[name_max] = pd.to_numeric(df[name_max],downcast='float')
    del block_max
    
    block_min = df.groupby(cols)[name].min()\
                        .reset_index().rename(columns={name:name_min})
    df = df.merge(block_min, on=cols, how='left')
    df[name_min].fillna(0,inplace=True)
    #df[name_min] = pd.to_numeric(df[name_min],downcast='float')
    del block_min
    
    block_std = df.groupby(cols)[name].std()\
                        .reset_index().rename(columns={name:name_std})
    df = df.merge(block_std, on=cols, how='left')
    df[name_std].fillna(0,inplace=True)
    #df[name_std] = pd.to_numeric(df[name_std],downcast='float')
    del block_std
    
    gc.collect()
    return df


training = add_block_units_stats(training, ['item_id'], 'item_cnt_block')
training = add_block_units_stats(training, ['shop_id'], 'shop_cnt_block')
training = add_block_units_stats(training, ['item_category_id'], 'category_cnt_block')
training = add_block_units_stats(training, ['shop_id', 'item_category_id'], 'shop_category_cnt_block')
#training = add_block_units_stats(training, ['shop_id', 'item_id','date_block_num'], 'shop_item_block')

item_cnt_block
   item_id  item_cnt_block_median
0  30       4.5                  
1  31       12.5                 
2  32       24.0                 
3  33       15.5                 
4  38       2.5                  
shop_cnt_block
   shop_id  shop_cnt_block_median
0  2        832.5                
1  3        734.5                
2  4        896.0                
3  5        1073.0               
4  6        1775.0               
category_cnt_block
   item_category_id  category_cnt_block_median
0  0                 0.0                      
1  2                 139.0                    
2  3                 803.5                    
3  5                 49.5                     
4  6                 275.5                    
shop_category_cnt_block
   shop_id  item_category_id  shop_category_cnt_block_median
0  2        0                 0.0                           
1  2        2                 3.0                           
2  2        3                 12.0                    

In [238]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,item_cnt_block_units,item_cnt_block_median,item_cnt_block_mean,item_cnt_block_max,item_cnt_block_min,item_cnt_block_std,shop_cnt_block_units,shop_cnt_block_median,shop_cnt_block_mean,shop_cnt_block_max,shop_cnt_block_min,shop_cnt_block_std,category_cnt_block_units,category_cnt_block_median,category_cnt_block_mean,category_cnt_block_max,category_cnt_block_min,category_cnt_block_std,shop_category_cnt_block_units,shop_category_cnt_block_median,shop_category_cnt_block_mean,shop_category_cnt_block_max,shop_category_cnt_block_min,shop_category_cnt_block_std
2019996,21123,50,30,0,61,0.0,1126.0,516.0,8.0,7,0,0.0,0.0,0.0,0.0,0.000000,53206400,1016.5,1040.0,1437.0,895.0,154.358886,49052072,654.0,787.1,2290.0,446.0,515.823768,11704,8.0,7.7,13.0,3.0,2.610457
1234286,13249,28,30,0,47,0.0,3612.0,630.0,45.0,7,0,0.0,0.0,0.0,0.0,0.000000,204645116,3766.5,4000.1,6879.0,2979.0,1036.085456,42005484,149.0,687.6,3421.0,0.0,1018.505680,40081,11.0,26.9,85.0,0.0,30.569874
377282,3989,14,26,0,56,0.0,1002.0,205.0,11.0,3,164,0.0,0.4,1.0,0.0,0.490496,53001760,1013.0,1036.0,1233.0,933.0,79.959521,2519122,192.0,198.2,312.0,85.0,74.355867,1488,5.0,4.8,11.0,1.0,3.492757
1797324,18653,46,28,0,40,0.0,1464.0,9208.0,156.0,5,82,0.0,0.2,1.0,0.0,0.400489,80438868,1558.0,1572.3,2100.0,1320.0,209.764797,3212026510,9293.5,10174.3,15109.0,6779.0,2444.994672,1364440,167.0,177.2,252.0,129.0,39.496362
1072000,11435,42,24,0,55,7.0,4125.0,9291.0,368.0,1,2091,5.0,5.1,8.0,2.0,1.702077,210103888,4129.0,4106.8,4648.0,3578.0,333.208239,1899903264,6413.5,6895.7,9291.0,4913.0,1348.041322,2184000,326.0,325.0,419.0,206.0,56.298061
1075831,11498,59,25,0,37,6.0,1206.0,4847.0,48.0,2,5535,10.0,13.5,28.0,3.0,9.426916,53671956,1023.0,1049.1,1293.0,790.0,160.835306,561635835,3908.5,4138.5,5959.0,2989.0,918.820587,179071,55.0,54.1,74.0,22.0,13.621529
1556002,16002,7,26,0,64,16.0,1430.0,1328.0,31.0,3,3567,4.0,8.7,36.0,0.0,10.634182,73609008,1333.5,1438.8,2084.0,1212.0,277.734167,61674496,991.0,1175.2,2211.0,801.0,424.519486,42112,32.0,32.9,61.0,12.0,13.951418
1712577,17714,2,31,0,49,1.0,942.0,571.0,5.0,8,1230,1.0,3.0,12.0,0.0,3.825663,43168808,832.5,843.8,987.0,727.0,77.605913,23335560,664.0,669.6,818.0,536.0,89.309960,5270,4.0,6.2,15.0,0.0,5.038836
1166302,12646,42,26,0,40,0.0,4133.0,13084.0,685.0,3,492,0.0,1.2,12.0,0.0,3.604398,210103888,4129.0,4106.8,4648.0,3578.0,333.208239,3212026510,9293.5,10174.3,15109.0,6779.0,2444.994672,4141830,496.0,537.9,702.0,430.0,92.699547
1614309,16227,21,33,3,65,48.0,1912.0,1518.0,53.0,10,35588,84.0,86.8,161.0,46.0,37.499196,93305608,1841.5,1823.8,2053.0,1622.0,124.217799,74668503,1879.0,2001.3,2941.0,1518.0,427.057020,47411,53.5,52.1,70.0,31.0,10.696529


In [239]:
training.describe()

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,item_cnt_block_units,item_cnt_block_median,item_cnt_block_mean,item_cnt_block_max,item_cnt_block_min,item_cnt_block_std,shop_cnt_block_units,shop_cnt_block_median,shop_cnt_block_mean,shop_cnt_block_max,shop_cnt_block_min,shop_cnt_block_std,category_cnt_block_units,category_cnt_block_median,category_cnt_block_mean,category_cnt_block_max,category_cnt_block_min,category_cnt_block_std,shop_category_cnt_block_units,shop_category_cnt_block_median,shop_category_cnt_block_mean,shop_category_cnt_block_max,shop_category_cnt_block_min,shop_category_cnt_block_std
count,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06
mean,1.101830e+04,3.156098e+01,2.850000e+01,2.445384e-01,4.627815e+01,1.257451e+01,1.699154e+03,3.773882e+03,8.389024e+01,5.500000e+00,5.155550e+03,8.791634e+00,1.257451e+01,4.066224e+01,3.116302e+00,1.149800e+01,8.692870e+07,1.633695e+03,1.699154e+03,2.342293e+03,1.339854e+03,3.034243e+02,8.311411e+08,3.444695e+03,3.773882e+03,5.851549e+03,2.520990e+03,9.875358e+02,4.513030e+05,8.002772e+01,8.389024e+01,1.289736e+02,5.351564e+01,2.269051e+01
std,6.246577e+03,1.773250e+01,2.872282e+00,1.080989e+00,1.676899e+01,8.034849e+01,1.394110e+03,3.751384e+03,1.724458e+02,2.872282e+00,2.534712e+04,5.266674e+01,6.182224e+01,1.920728e+02,2.927392e+01,5.008119e+01,6.728722e+07,1.260525e+03,1.315231e+03,1.902227e+03,1.074559e+03,3.487790e+02,1.170224e+09,3.245611e+03,3.522610e+03,5.108302e+03,2.405506e+03,8.299934e+02,1.143970e+06,1.520227e+02,1.616670e+02,2.734789e+02,1.077633e+02,5.558359e+01
min,3.000000e+01,2.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,-1.000000e+00,3.830000e+02,-1.000000e+00,-1.000000e+00,1.000000e+00,-4.100000e+01,0.000000e+00,-1.000000e-01,0.000000e+00,-1.000000e+00,0.000000e+00,2.178904e+07,4.220000e+02,4.259000e+02,4.840000e+02,3.830000e+02,3.450961e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,-4.000000e+00,0.000000e+00,-1.000000e-01,0.000000e+00,-1.000000e+00,0.000000e+00
25%,5.394250e+03,1.600000e+01,2.600000e+01,0.000000e+00,3.700000e+01,0.000000e+00,9.320000e+02,6.800000e+02,4.000000e+00,3.000000e+00,4.100000e+02,0.000000e+00,1.000000e+00,4.000000e+00,0.000000e+00,1.184662e+00,5.169718e+07,9.890000e+02,1.010500e+03,1.247000e+03,7.900000e+02,1.242178e+02,3.709315e+07,6.540000e+02,7.871000e+02,1.622000e+03,4.460000e+02,3.046296e+02,4.023000e+03,3.500000e+00,4.600000e+00,1.200000e+01,0.000000e+00,3.130728e+00
50%,1.119050e+04,3.400000e+01,2.850000e+01,0.000000e+00,4.300000e+01,2.000000e+00,1.272000e+03,2.203000e+03,3.300000e+01,5.500000e+00,1.722000e+03,2.000000e+00,4.200000e+00,1.100000e+01,0.000000e+00,3.138314e+00,6.666660e+07,1.318500e+03,1.303100e+03,1.715000e+03,9.900000e+02,1.843054e+02,1.802443e+08,1.917000e+03,1.880900e+03,4.824000e+03,1.316000e+03,9.188206e+02,4.686500e+04,3.150000e+01,3.400000e+01,6.000000e+01,1.700000e+01,1.218249e+01
75%,1.606725e+04,4.700000e+01,3.100000e+01,0.000000e+00,5.800000e+01,1.000000e+01,1.740000e+03,6.353000e+03,1.020000e+02,8.000000e+00,4.756000e+03,8.500000e+00,1.160000e+01,3.000000e+01,2.000000e+00,8.594192e+00,9.245635e+07,1.699500e+03,1.807200e+03,2.329000e+03,1.320000e+03,2.777342e+02,1.899903e+09,6.413500e+03,6.895700e+03,9.291000e+03,4.913000e+03,1.348041e+03,4.928000e+05,1.015000e+02,1.043000e+02,1.530000e+02,6.800000e+01,2.467247e+01
max,2.216700e+04,5.900000e+01,3.300000e+01,2.000000e+01,8.300000e+01,7.869000e+03,8.815000e+03,1.510900e+04,5.129000e+03,1.000000e+01,1.582846e+06,3.512000e+03

In [240]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'shop_item_cnt_block',
       'item_category_id', 'item_cnt_block', 'shop_cnt_block',
       'category_cnt_block', 'shop_category_cnt_block', 'month',
       'item_cnt_block_units', 'item_cnt_block_median',
       'item_cnt_block_mean', 'item_cnt_block_max', 'item_cnt_block_min',
       'item_cnt_block_std', 'shop_cnt_block_units',
       'shop_cnt_block_median', 'shop_cnt_block_mean',
       'shop_cnt_block_max', 'shop_cnt_block_min', 'shop_cnt_block_std',
       'category_cnt_block_units', 'category_cnt_block_median',
       'category_cnt_block_mean', 'category_cnt_block_max',
       'category_cnt_block_min', 'category_cnt_block_std',
       'shop_category_cnt_block_units', 'shop_category_cnt_block_median',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_max',
       'shop_category_cnt_block_min', 'shop_category_cnt_block_std'],
      dtype=object)

In [261]:
features = [
    
'item_cnt_block', 'shop_cnt_block',
       'category_cnt_block', 'shop_category_cnt_block', 
       'item_cnt_block_units', 'item_cnt_block_median',
       'item_cnt_block_mean', 'item_cnt_block_max', 'item_cnt_block_min',
       'item_cnt_block_std', 'shop_cnt_block_units',
       'shop_cnt_block_median', 'shop_cnt_block_mean',
       'shop_cnt_block_max', 'shop_cnt_block_min', 'shop_cnt_block_std',
       'category_cnt_block_units', 'category_cnt_block_median',
       'category_cnt_block_mean', 'category_cnt_block_max',
       'category_cnt_block_min', 'category_cnt_block_std',
       'shop_category_cnt_block_units', 'shop_category_cnt_block_median',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_max',
       'shop_category_cnt_block_min', 'shop_category_cnt_block_std'

]

In [243]:
training.fillna(0,inplace=True)
training = training.sample(frac=1).reset_index(drop=True)


In [20]:
gc.collect()

ZEROS_KEEP=0.25

#x_train = training[(training['date_block_num'] < 33) & (training['val_ignore'] == False)]
x_train = training[(training['date_block_num'] < 33)]
y_train = x_train['item_cnt_block']





x_val = training[training['date_block_num'] == 33]
y_val = x_val['item_cnt_block']

pos_val_len = len(y_val[y_val != 0])
print("pos_val_len", pos_val_len)

zeros_keep_indices_val = y_val[y_val == 0].sample(int(pos_val_len/ZEROS_KEEP)).index
print("zeros_keep_indices_val", len(zeros_keep_indices_val))
non_zeros_val_indices = y_val[y_val != 0].index
print("non_zeros_val_indices", len(non_zeros_val_indices))

val_indices = np.append(np.array(zeros_keep_indices_val), np.array(non_zeros_val_indices))

y_val = y_val.loc[val_indices]
x_val = x_val.loc[val_indices]



pos_val_len 29202
zeros_keep_indices_val 116808
non_zeros_val_indices 29202


In [263]:
training[features].sample(10)

,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_cnt_block_units,item_cnt_block_median,item_cnt_block_mean,item_cnt_block_max,item_cnt_block_min,item_cnt_block_std,shop_cnt_block_units,shop_cnt_block_median,shop_cnt_block_mean,shop_cnt_block_max,shop_cnt_block_min,shop_cnt_block_std,category_cnt_block_units,category_cnt_block_median,category_cnt_block_mean,category_cnt_block_max,category_cnt_block_min,category_cnt_block_std,shop_category_cnt_block_units,shop_category_cnt_block_median,shop_category_cnt_block_mean,shop_category_cnt_block_max,shop_category_cnt_block_min,shop_category_cnt_block_std
1022524,-0.131608,-0.230365,0.687511,0.139811,-0.095023,-0.062499,-0.095023,-0.138813,-0.038133,-0.152651,0.082150,0.052204,0.082150,0.092895,-0.019407,0.333591,0.913297,0.914714,0.886223,0.673306,0.994390,0.434348,0.391473,0.364895,0.308720,0.168300,0.487034,0.002127
1783145,-0.069379,-0.332939,0.597944,0.203599,-0.075612,-0.024525,-0.075612,-0.133607,-0.038133,-0.164837,-0.197953,-0.238151,-0.197953,-0.135784,-0.118983,-0.073657,0.913297,0.914714,0.886223,0.673306,0.994390,0.434348,0.420257,0.384629,0.339029,0.182926,0.375679,0.018011
678648,-0.119162,-0.309985,-0.777815,-0.486473,-0.196928,-0.166930,-0.196928,-0.196083,-0.106453,-0.211264,-0.218558,-0.228234,-0.218558,-0.155761,-0.090133,-0.225880,-0.556216,-0.723345,-0.624021,-0.201153,-0.761998,0.257047,-0.394506,-0.526420,-0.518908,-0.471603,-0.496604,-0.408223
1647754,-0.144054,-0.275555,1.448564,1.340188,-0.177517,-0.128955,-0.177517,-0.196083,-0.106453,-0.205597,-0.136367,-0.152076,-0.136367,0.053468,-0.135734,0.124744,2.034555,1.802066,1.816954,1.812237,1.770110,1.755989,2.134984,1.795602,1.805624,1.843018,1.582027,1.660505
708867,-0.007150,0.775295,0.599277,1.606939,-0.015763,0.070412,-0.015763,-0.102369,-0.072293,-0.122972,0.853574,0.774522,0.853574,0.946632,0.861886,0.814572,0.913297,0.914714,0.886223,0.673306,0.994390,0.434348,1.074064,1.029269,1.027481,0.848426,1.396435,0.346166
1516929,0.229320,-0.538088,0.076537,-0.190728,0.194517,0.260285,0.194517,0.053822,0.235148,0.044558,-0.501170,-0.489633,-0.501170,-0.475912,-0.413987,-0.427392,-0.230302,0.142902,0.103508,0.021034,0.194558,-0.082790,-0.236235,-0.164631,-0.180558,-0.190046,-0.181097,-0.160322
106064,0.017741,-0.672224,-0.693313,-0.335701,0.097465,0.117880,0.097465,0.032997,0.132668,-0.014601,-0.721435,-0.720886,-0.721435,-0.716157,-0.668045,-0.579071,-0.672200,-0.716104,-0.746884,-0.814273,-0.736640,-0.841343,-0.379475,-0.411305,-0.406949,-0.376532,-0.394528,-0.324778
885147,0.030187,0.198583,-0.613875,-0.486473,0.073202,0.146361,0.073202,0.048616,-0.106453,0.042252,1.116037,0.555566,1.116037,2.378638,0.296072,4.318132,-0.649145,-0.470696,-0.550240,-0.681743,-0.622319,-0.741041,-0.394506,-0.526420,-0.518908,-0.471603,-0.496604,-0.408223
1491877,-0.119162,-0.020912,-0.529373,-0.248717,-0.111198,-0.090980,-0.111198,-0.123194,-0.072293,-0.138827,-0.096450,-0.060050,-0.096450,-0.127373,-0.018476,-0.268535,-0.623913,-0.466074,-0.537381,-0.645332,-0.500930,-0.697567,-0.349731,-0.296191,-0.277053,-0.186389,-0.273893,-0.149725
1588765,-0.081825,-0.736781,-0.766886,-0.480674,-0.017381,-0.015031,-0.017381,-0.050305,0.064347,-0.073381,-0.747666,-0.713350,-0.747666,-0.717734,-0.749009,-0.557614,-0.657538,-0.756004,-0.737715,-0.712673,-0.715023,-0.678338,-0.387904,-0.490241,-0.482413,-0.420411,-0.487324,-0.344616


In [262]:
from sklearn import preprocessing


training[features] = preprocessing.scale(training[features])


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint32, int64, float64 were all converted to float64 by the scale function.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [275]:
a = training[['shop_item_cnt_block'] + ['date_block_num','item_id', 'shop_id'] + features].sort_values(by=["date_block_num"]).groupby(["item_id", "shop_id"])#['item_id_mean_encoding']#[features+['item_cnt_block']]

In [276]:
#groups
lstm_data = []
lstm_y = []

for name, group in a:
    #print(group.values)
    steps = []
    ys = []
    #print("group.values",group.values)
    for step in group.values:
        #print("step", len(step))
        #step is np.array 
        #step[4:] is np.array print(type(step[4:]))
        steps.append(step[4:])
        #print(step[9])
        #print(step[0])
        ys.append(step[0])
    #remove last
    #print(type(steps[0:8]))
    lstm_data.append(np.array(steps[0:8]))
    #remove first
    #print(ys)
    lstm_y.append(ys[1:9][-1])
    
lstm_y = np.array([np.array([y]) for y in lstm_y])

In [36]:
np.array(np.array(lstm_data)).shape

(271148, 8, 33)

In [38]:
np.array(lstm_y).shape

(271148,)

In [279]:
small_data = np.array(lstm_data[0:500000])
small_y = np.array(lstm_y[0:500000])

In [280]:
from sklearn.model_selection import train_test_split

train_data, val_data, y_train, y_val = train_test_split(small_data, small_y, test_size=0.33, random_state=42)

In [271]:
features

['item_cnt_block',
 'shop_cnt_block',
 'category_cnt_block',
 'shop_category_cnt_block',
 'item_cnt_block_units',
 'item_cnt_block_median',
 'item_cnt_block_mean',
 'item_cnt_block_max',
 'item_cnt_block_min',
 'item_cnt_block_std',
 'shop_cnt_block_units',
 'shop_cnt_block_median',
 'shop_cnt_block_mean',
 'shop_cnt_block_max',
 'shop_cnt_block_min',
 'shop_cnt_block_std',
 'category_cnt_block_units',
 'category_cnt_block_median',
 'category_cnt_block_mean',
 'category_cnt_block_max',
 'category_cnt_block_min',
 'category_cnt_block_std',
 'shop_category_cnt_block_units',
 'shop_category_cnt_block_median',
 'shop_category_cnt_block_mean',
 'shop_category_cnt_block_max',
 'shop_category_cnt_block_min',
 'shop_category_cnt_block_std']

In [283]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


#x_train_scaled = MinMaxScaler().fit_transform(x_train[features])


#x_reshaped = np.reshape(x_train_scaled, (x_train_scaled.shape[0], 10, x_train_scaled.shape[1]))
    
#x_val_scaled_reshaped = np.reshape(x_val_scaled, (x_val_scaled.shape[0], 1, x_val_scaled.shape[1]))


callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, verbose=0)
]


my_model = Sequential()
my_model.add(LSTM(units = 16,input_shape = (8,28)))
my_model.add(BatchNormalization())
my_model.add(Dropout(0.5))
#my_model.add(BatchNormalization())
#my_model.add(LSTM(units = 16))
#my_model.add(BatchNormalization())
#my_model.add(Flatten())
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()


#lstmd_dataa = np.array(np.array(lstm_data)).reshape(271148,8,51)
#print(lstmd_dataa.shape)
#print(lstmd_dataa)
#lstm_yy = np.array([np.array([y]) for y in lstm_y[0:100]])
#print(lstm_yy.shape)
#print(lstm_yy)

history = my_model.fit(train_data, y_train, batch_size=4096, epochs=100,
                      validation_data=(val_data,y_val), callbacks=callbacks
                      )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_46 (LSTM)               (None, 16)                2880      
_________________________________________________________________
batch_normalization_23 (Batc (None, 16)                64        
_________________________________________________________________
dropout_20 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 17        
Total params: 2,961
Trainable params: 2,929
Non-trainable params: 32
_________________________________________________________________
Train on 140536 samples, validate on 69220 samples
Epoch 1/100
140536/140536 [==============================] - 12s 83us/step - loss: 3.7726 - mean_squared_error: 3.7726 - val_loss: 1.1631 - val_mean_squared_error: 1.1631
Epoch 2/100
140536/140536 [======================

KeyboardInterrupt: 

In [149]:
lstmd_dataa.shape

(8, 49)

In [ ]:
features = [
    
'item_id_mean_encoding',
       'shop_id_mean_encoding', #'item_category_id_mean_encoding',
#       'month_mean_encoding', 'shop_cat_mean_encoding',
#       'shop_item_mean_encoding', 'date_block_num_mean_encoding',
 
#       'item_block_units_rolling_3', 'item_block_mean_rolling_3',
#       'item_block_median_rolling_3', 'item_block_min_rolling_3',
#       'item_block_max_rolling_3', 'item_block_std_rolling_3',
    
#       'shop_block_units_rolling_3', 'shop_block_mean_rolling_3',
#       'shop_block_median_rolling_3', 'shop_block_min_rolling_3',
#       'shop_block_max_rolling_3', 'shop_block_std_rolling_3',
    
#       'cat_block_units_rolling_3', 'cat_block_mean_rolling_3',
#       'cat_block_median_rolling_3', 'cat_block_min_rolling_3',
#       'cat_block_max_rolling_3', 'cat_block_std_rolling_3',
    
       'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3',
       'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3',
       'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3',
    
    
#       'item_block_units_lag_1', 'item_block_mean_lag_1',
#       'item_block_median_lag_1', 'item_block_min_lag_1',
#       'item_block_max_lag_1', 'item_block_std_lag_1',
    
#       'shop_block_units_lag_1', 'shop_block_mean_lag_1',
#       'shop_block_median_lag_1', 'shop_block_min_lag_1',
       'shop_block_max_lag_1', 'shop_block_std_lag_1',
    
#      'cat_block_units_lag_1', 'cat_block_mean_lag_1',
#       'cat_block_median_lag_1', 'cat_block_min_lag_1',
#       'cat_block_max_lag_1', 'cat_block_std_lag_1',
    
       'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1',
#       'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1',
#       'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1',
    
       'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1',
#       'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1',
#       'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1',
    
       'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1',
    
    'shop_pred', 'shop_cat_pred', 'cat_pred'

]


In [60]:
features = ['item_id_mean_encoding',
 'shop_cat_mean_encoding',
 'shop_item_block_units_lag_1',
 'shop_cat_pred',
 'shop_cat_block_units_lag_1',
 'cat_pred',
 'comp2_lag_1',
 'shop_cat_block_max_rolling_3',
 'shop_block_mean_lag_1',
 'item_share_block_lag_1',
 'shop_item_block_mean_lag_1']

In [40]:
from sklearn.preprocessing import MinMaxScaler

x_train_scaled = MinMaxScaler().fit_transform(x_train[features])
x_val_scaled = MinMaxScaler().fit_transform(x_val[features])

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [63]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor


lr_model =  BayesianRidge()
lr_model.fit(x_train_scaled, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val_scaled)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

rmse:  1.0587979027152195


In [ ]:
lr_model.coef_

In [45]:
#test            = pd.read_csv('test.csv.gz')
test = test.set_index('item_id').join(items.set_index('item_id'))
test.reset_index(inplace=True)

In [46]:
def predict(shop_id):
    return shop_models[shop_id].predict([[34]])[0][0]

#test['shop_pred'] = test.apply(lambda row: predict(row['shop_id']), axis=1)

def predict(shop_id, cat_id):
    if shop_id in shop_cat_models and cat_id in shop_cat_models[shop_id]:
        return shop_cat_models[shop_id][cat_id].predict([[34]])[0][0]

test['shop_cat_pred'] = test.apply(lambda row: predict(row['shop_id'],row['item_category_id']), axis=1)

def predict(cat_id):
    if cat_id in cat_models:
        return cat_models[cat_id].predict([[34]])[0][0]

test['cat_pred'] = test.apply(lambda row: predict(row['item_category_id']), axis=1)

In [47]:
item_features = [ 
    'item_id_mean_encoding'
                ]

merge_col = ['item_id']
cols=item_features+merge_col

test = test.merge(training.drop_duplicates('item_id')[cols], on=merge_col, how='left')

In [ ]:
shop_features = [
        'shop_id_mean_encoding'
]

merge_col = ['shop_id']
cols=shop_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [ ]:
cat_features = [
        'item_category_id_mean_encoding'#,'cat_me_real'
]

merge_col = ['item_category_id']
cols=cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [ ]:
shop_item_features = [
        'shop_item_mean_encoding'
]

merge_col = ['shop_id', 'item_id']
cols=shop_item_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [53]:
shop_cat_features = [
        'shop_cat_mean_encoding'
]

merge_col = ['shop_id', 'item_id']
cols=shop_cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [48]:
def add_rolls_test(df, cols, name, rolls = [3]):
    for roll in rolls:
        print(name, roll)
        roll_name = name+"_rolling_" + str(roll)
        roll_name_tmp = roll_name + "_tmp"
        
        try:
            df.drop(columns=[roll_name],inplace=True)
        except:
            pass       

    
        block_units_rolling_temp = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].rolling(roll,min_periods=2).mean().reset_index()\
            .rename(columns={name:roll_name})\
            [cols+[roll_name]]
        
        print([cols[0:len(cols)-1]+[roll_name]])
        thirty_three = block_units_rolling_temp[block_units_rolling_temp['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1]+[roll_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')
    

        del block_units_rolling_temp
        gc.collect()
        

    
    return df
    

#test = add_rolls_test(test, ['item_id','date_block_num'], 'item_block_mean')
#test = add_rolls_test(test, ['item_id','date_block_num'], 'item_block_units')
#test = add_rolls_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')

#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_median')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_min')
test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_max')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_std')

shop_cat_block_max 3
[['shop_id', 'item_category_id', 'shop_cat_block_max_rolling_3']]


In [ ]:
test = add_rolls_test(test, ['item_category_id','date_block_num'], 'cat_block_mean')

In [ ]:
test = add_rolls_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

In [49]:
def add_lags_test(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()
        
        thirty_three = result[result['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1] + [lag_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')

        gc.collect()
    
    return df
                                         

                                        
#test = add_lags_test(test, ['item_id','date_block_num'], 'item_block_mean')
#test = add_lags_test(test, ['item_id','date_block_num'], 'item_block_units')
test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
#test = add_lags_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')

#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_max')
#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_std')

test = add_lags_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')

test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')
test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

test = add_lags_test(test, ['item_id','date_block_num'], 'item_share_block')
#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_share_block')
#test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'comp2')

shop_block_mean 1
shop_cat_block_units 1
shop_item_block_units 1
shop_item_block_mean 1
item_share_block 1


In [ ]:
test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')
#test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')

In [50]:
#training['comp2'] = training['item_share_block'] * training['shop_share_block']


test = add_lags_test(test, ['item_id','shop_id','date_block_num'], 'comp2')

comp2 1


In [54]:
test.fillna(0, inplace=True)

In [55]:
from sklearn.preprocessing import MinMaxScaler

test_scaled = MinMaxScaler().fit_transform(test[features])


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [64]:
preds = lr_model.predict(test_scaled)
preds.clip(0,20,out=preds)


array([ 0.        ,  0.        ,  0.15157035, ...,  0.26322811,
        0.28951454,  0.21382811])

In [92]:
preds = lg_model.predict(test[features])
preds.clip(0,20,out=preds)


array([ 0.11804522,  0.07073731,  0.18383498, ...,  0.04995914,
        0.28776269,  0.09947505])

In [96]:

print(np.mean(preds))
print(np.max(preds))

0.291327336924
17.0728554221


In [ ]:
preds = np.mean(np.array([lg_preds, lr_preds]),axis=0)

In [97]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)

In [95]:
lr_preds = pd.read_csv('lr110.csv')['item_cnt_month']
lg_preds = pd.read_csv('lg110.csv')['item_cnt_month']
#cb_preds = pd.read_csv('cb102.csv')['item_cnt_month']


#preds = np.mean(np.array([lr_preds, lg_preds]),axis=0)

preds = (lg_preds * 0.50) + (lr_preds * 0.50)

In [ ]:
training.columns.values

In [30]:
all_features = ['item_id_mean_encoding',
       'shop_id_mean_encoding', 'item_category_id_mean_encoding',
       'month_mean_encoding', 'shop_cat_mean_encoding',
       'shop_item_mean_encoding', 'date_block_num_mean_encoding',
       'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3',
       'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3',
       'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3',
       'shop_block_units_lag_1', 'shop_block_mean_lag_1',
       'shop_block_median_lag_1', 'shop_block_min_lag_1',
       'shop_block_max_lag_1', 'shop_block_std_lag_1',
       'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1',
       'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1',
       'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1',
       'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1',
       'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1',
       'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1',
       'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1',
       'shop_share', 'item_share', 'shop_pred', 'shop_cat_pred',
       'cat_pred']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Create a random forest classifier
clf = RandomForestRegressor(n_estimators=10, random_state=0, n_jobs=8)

# Train the classifier
clf.fit(x_train[all_features], y_train)

# Print the name and gini importance of each feature


In [ ]:
f_ = sorted(list(zip(all_features, clf.feature_importances_)), key=lambda x: x[1])[::-1][0:]

features = [f[0] for f in f_]

from sklearn.preprocessing import MinMaxScaler

x_train_scaled = MinMaxScaler().fit_transform(x_train[features])
x_val_scaled = MinMaxScaler().fit_transform(x_val[features])


lr_model =  Ridget(alpha=0)
lr_model.fit(x_train_scaled, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val_scaled)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

In [ ]:
sorted(list(zip(all_features, clf.feature_importances_)), key=lambda x: x[1])[::-1]

In [ ]:
all_features

In [38]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor
import math

all_f = all_features.copy()
keep = []

print(all_f)

best = 10000
while len(all_f) > 0:
    print("keep", keep)
    len_keep = len(keep)
    #print(len(all_f))
    res = {}
    for idx,feat in enumerate(all_f):

        potential = keep + [feat]

        x_train_scaled = MinMaxScaler().fit_transform(x_train[potential])
        x_val_scaled = MinMaxScaler().fit_transform(x_val[potential])


        lr_model =  Ridge(alpha=0)
        lr_model.fit(x_train_scaled, y_train)


        lr_val_preds = lr_model.predict(x_val_scaled)
        lr_val_preds.clip(0,20,out=lr_val_preds)
        rms = math.sqrt(mean_squared_error(y_val, lr_val_preds))

        res[feat] = rms
        
    sorted_by_value = sorted(res.items(), key=lambda kv: kv[1])
    best_score = sorted_by_value[0][1]
    best_feature = sorted_by_value[0][0]
    print("best_score", best_score)
    print("best_feature", best_feature)
    if best_score < best:
        best = best_score
        keep.append(best_feature)
        all_f.remove(best_feature)
    else:
        break
    
keep

['item_id_mean_encoding', 'shop_id_mean_encoding', 'item_category_id_mean_encoding', 'month_mean_encoding', 'shop_cat_mean_encoding', 'shop_item_mean_encoding', 'date_block_num_mean_encoding', 'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3', 'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3', 'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3', 'shop_block_units_lag_1', 'shop_block_mean_lag_1', 'shop_block_median_lag_1', 'shop_block_min_lag_1', 'shop_block_max_lag_1', 'shop_block_std_lag_1', 'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1', 'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1', 'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1', 'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1', 'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1', 'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1', 'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1', 'shop_share', 'item_share', 'shop_pred

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return

best_score 1.2213909415288133
best_feature item_id_mean_encoding
keep ['item_id_mean_encoding']


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were

...

best_score 1.0587356513519661
best_feature shop_item_block_mean_lag_1
keep ['item_id_mean_encoding', 'shop_cat_mean_encoding', 'shop_item_block_units_lag_1', 'shop_cat_pred', 'shop_cat_block_units_lag_1', 'cat_pred', 'comp2_lag_1', 'shop_cat_block_max_rolling_3', 'shop_block_mean_lag_1', 'item_share_block_lag_1', 'shop_item_block_mean_lag_1']


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataCo

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataCo

best_score 1.0587553332327528
best_feature shop_share_block_lag_1


['item_id_mean_encoding',
 'shop_cat_mean_encoding',
 'shop_item_block_units_lag_1',
 'shop_cat_pred',
 'shop_cat_block_units_lag_1',
 'cat_pred',
 'comp2_lag_1',
 'shop_cat_block_max_rolling_3',
 'shop_block_mean_lag_1',
 'item_share_block_lag_1',
 'shop_item_block_mean_lag_1']

In [ ]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_by_value = sorted(x.items(), key=lambda kv: kv[1])[::-1]
sorted_by_value

In [91]:
gc.collect()
lgtrain = lgbm.Dataset(x_train[features], label=y_train)
lgval = lgbm.Dataset(x_val[features], label=y_val)


params = {
        "num_threads": 16,
        #"device": "gpu",
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        "max_bin": 30,#default 255
        #"num_leaves": 70, #default 31
        "bagging_fraction": 0.5,
        "bagging_freq": 1,
        "min_data_in_leaf": 2500,
        "feature_fraction": 0.7,
        #"lambda_l2": 3,
        #"max_depth": 7,
        #"min_gain_to_split": 10,
        "learning_rate" : 0.1,
        #"histogram_pool_size": 1000,
        #"categorical_column": [0,1,2,3,4]
}

evals_result = {}
lg_model = lgbm.train(params, lgtrain, 20000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=10, 
                      verbose_eval=10, 
                      evals_result=evals_result)

scores = {}
for i,score in enumerate(lg_model.feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's rmse: 1.16893
[20]	valid_0's rmse: 1.12071
[30]	valid_0's rmse: 1.10703
[40]	valid_0's rmse: 1.10052
[50]	valid_0's rmse: 1.09479
[60]	valid_0's rmse: 1.08897
[70]	valid_0's rmse: 1.08548
[80]	valid_0's rmse: 1.08165
[90]	valid_0's rmse: 1.07856
[100]	valid_0's rmse: 1.07572
[110]	valid_0's rmse: 1.07395
[120]	valid_0's rmse: 1.0719
[130]	valid_0's rmse: 1.07138
Early stopping, best iteration is:
[123]	valid_0's rmse: 1.07102


[('shop_cat_mean_encoding', 579),
 ('shop_cat_pred', 538),
 ('item_id_mean_encoding', 532),
 ('cat_pred', 525),
 ('shop_block_mean_lag_1', 398),
 ('shop_item_block_units_lag_1', 360),
 ('shop_cat_block_max_rolling_3', 333),
 ('shop_cat_block_units_lag_1', 307),
 ('shop_item_block_mean_lag_1', 118),
 ('item_share_block_lag_1', 0),
 ('comp2_lag_1', 0)]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor()

neigh.fit(x_train_scaled, y_train) 